In [ ]:
import os
from keras.layers.advanced_activations import ELU
weg = '/media/n01z3/storage3/datasets/carvana/'
os.environ["CUDA_VISIBLE_DEVICES"]="2"
from keras.layers import Input, merge, concatenate, MaxPooling2D, UpSampling2D, Cropping2D, ZeroPadding2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import downscale_local_mean
from os.path import join
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D,MaxPool2D,MaxPool1D,Conv3D,MaxPool3D,MaxPooling2D,MaxPooling3D,Dense,Flatten,Reshape
from keras.layers import Conv2DTranspose
from keras.models import Sequential
import keras.backend as K
import gc
from tqdm import tqdm_notebook
from scipy.misc import imresize
df_mask = pd.read_csv(weg+'train_masks.csv', usecols=['img'])
ids_train = df_mask['img'].map(lambda s: s.split('_')[0]).unique()
from keras.metrics import binary_accuracy
from keras.models import load_model
imgs_idx = list(range(1, 17))
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, BatchNormalization, Activation, UpSampling2D
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, BatchNormalization, Activation, UpSampling2D
from keras.optimizers import SGD
import keras.backend as K
from skimage.io import imread


In [ ]:
load_img = lambda im: cv2.imread(join(weg+'train', '{}.jpg'.format(im)))
load_mask = lambda im: imread(join(weg+'train_masks', '{}_mask.gif'.format(im)))
load_mask_gif = lambda im: imread(join(weg+'train_masks', '{}_mask.gif'.format(im)))[:,:,0]

resize = lambda im: downscale_local_mean(im, (4,4) if im.ndim==2 else (4,4,1))
mask_image = lambda im, mask: (im * np.expand_dims(mask, 2))
from os.path import join

In [ ]:
from keras.optimizers import Adam
from keras.losses import binary_crossentropy

smooth = 1.

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


In [ ]:
from renorm import BatchRenormalization

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def get_unet_1024(input_shape=(1280, 1280, 3),
                 num_classes=1):
    inputs = Input(shape=input_shape)
    # 512

    down0a = Conv2D(16, (3, 3), padding='same')(inputs)
    down0a = BatchRenormalization()(down0a)
    down0a = ELU()(down0a)
    down0a = Conv2D(16, (3, 3), padding='same')(down0a)
    down0a = BatchRenormalization()(down0a)
    down0a = ELU()(down0a)
    down0a_pool = MaxPooling2D((2, 2), strides=(2, 2))(down0a)
    # 256

    down0 = Conv2D(32, (3, 3), padding='same')(down0a_pool)
    down0 = BatchRenormalization()(down0)
    down0 = ELU()(down0)
    down0 = Conv2D(32, (3, 3), padding='same')(down0)
    down0 = BatchRenormalization()(down0)
    down0 = ELU()(down0)
    down0_pool = MaxPooling2D((2, 2), strides=(2, 2))(down0)
    # 128

    down1 = Conv2D(64, (3, 3), padding='same')(down0_pool)
    down1 = BatchRenormalization()(down1)
    down1 = ELU()(down1)
    down1 = Conv2D(64, (3, 3), padding='same')(down1)
    down1 = BatchRenormalization()(down1)
    down1 = ELU()(down1)
    down1_pool = MaxPooling2D((2, 2), strides=(2, 2))(down1)
    # 64

    down2 = Conv2D(128, (3, 3), padding='same')(down1_pool)
    down2 = BatchRenormalization()(down2)
    down2 = ELU()(down2)
    down2 = Conv2D(128, (3, 3), padding='same')(down2)
    down2 = BatchRenormalization()(down2)
    down2 = ELU()(down2)
    down2_pool = MaxPooling2D((2, 2), strides=(2, 2))(down2)
    # 32

    down3 = Conv2D(256, (3, 3), padding='same')(down2_pool)
    down3 = BatchRenormalization()(down3)
    down3 = ELU()(down3)
    down3 = Conv2D(256, (3, 3), padding='same')(down3)
    down3 = BatchRenormalization()(down3)
    down3 = ELU()(down3)
    down3_pool = MaxPooling2D((2, 2), strides=(2, 2))(down3)
    # 16

    down4 = Conv2D(512, (3, 3), padding='same')(down3_pool)
    down4 = BatchRenormalization()(down4)
    down4 = ELU()(down4)
    down4 = Conv2D(512, (3, 3), padding='same')(down4)
    down4 = BatchRenormalization()(down4)
    down4 = ELU()(down4)
    down4_pool = MaxPooling2D((2, 2), strides=(2, 2))(down4)
    # 8

    center = Conv2D(1024, (3, 3), padding='same')(down4_pool)
    center = BatchRenormalization()(center)
    center = ELU()(center)
    center = Conv2D(1024, (3, 3), padding='same')(center)
    center = BatchRenormalization()(center)
    center = ELU()(center)
    # center

    up4 = UpSampling2D((2, 2))(center)
    up4 = concatenate([down4, up4], axis=3)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchRenormalization()(up4)
    up4 = ELU()(up4)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchRenormalization()(up4)
    up4 = ELU()(up4)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchRenormalization()(up4)
    up4 = ELU()(up4)
    # 16

    up3 = UpSampling2D((2, 2))(up4)
    up3 = concatenate([down3, up3], axis=3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchRenormalization()(up3)
    up3 = ELU()(up3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchRenormalization()(up3)
    up3 = ELU()(up3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchRenormalization()(up3)
    up3 = ELU()(up3)
    # 32

    up2 = UpSampling2D((2, 2))(up3)
    up2 = concatenate([down2, up2], axis=3)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchRenormalization()(up2)
    up2 = ELU()(up2)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchRenormalization()(up2)
    up2 = ELU()(up2)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchRenormalization()(up2)
    up2 = ELU()(up2)
    # 64

    up1 = UpSampling2D((2, 2))(up2)
    up1 = concatenate([down1, up1], axis=3)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchRenormalization()(up1)
    up1 = ELU()(up1)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchRenormalization()(up1)
    up1 = ELU()(up1)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchRenormalization()(up1)
    up1 = ELU()(up1)
    # 128

    up0 = UpSampling2D((2, 2))(up1)
    up0 = concatenate([down0, up0], axis=3)
    up0 = Conv2D(32, (3, 3), padding='same')(up0)
    up0 = BatchRenormalization()(up0)
    up0 = ELU()(up0)
    up0 = Conv2D(32, (3, 3), padding='same')(up0)
    up0 = BatchRenormalization()(up0)
    up0 = ELU()(up0)
    up0 = Conv2D(32, (3, 3), padding='same')(up0)
    up0 = BatchRenormalization()(up0)
    up0 = ELU()(up0)
    # 256

    up0a = UpSampling2D((2, 2))(up0)
    up0a = concatenate([down0a, up0a], axis=3)
    up0a = Conv2D(16, (3, 3), padding='same')(up0a)
    up0a = BatchRenormalization()(up0a)
    up0a = ELU()(up0a)
    up0a = Conv2D(16, (3, 3), padding='same')(up0a)
    up0a = BatchRenormalization()(up0a)
    up0a = ELU()(up0a)
    up0a = Conv2D(16, (3, 3), padding='same')(up0a)
    up0a = BatchRenormalization()(up0a)
    up0a = ELU()(up0a)
    # 512

    classify = Conv2D(num_classes, (1, 1), activation='sigmoid')(up0a)

    model = Model(inputs=inputs, outputs=classify)

    model.compile(optimizer=SGD(lr=0.01, momentum=0.9), loss='binary_crossentropy', metrics=[dice_loss])

    return model

In [ ]:
import pandas as pd
import numpy as np

import cv2
from tqdm import tqdm


df_test = pd.read_csv(weg+'sample_submission.csv')
ids_test = df_test['img'].map(lambda s: s.split('.')[0])

input_size = 1280
input_size_2 = 1280
batch_size = 1

orig_width = 1918
orig_height = 1280

threshold = 0.5

model = get_unet_1024()
model.load_weights(filepath='unet_new_1280_1280_2_batch_renorm_9_my_gen_ELU.hdf5')

names = []
for id in ids_test:
    names.append('{}.jpg'.format(id))


# https://www.kaggle.com/stainsby/fast-tested-rle
def run_length_encode(mask):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    inds = mask.flatten()
    runs = np.where(inds[1:] != inds[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    rle = ' '.join([str(r) for r in runs])
    return rle


rles = []

test_splits = 8*59  # Split test set (number of splits must be multiple of 2)
ids_test_splits = np.split(ids_test, indices_or_sections=test_splits)



In [ ]:
def _mask_to_rle_string(mask):
    """Convert boolean/`binary uint` mask to RLE string."""
    # Mask to RLE
    pixels = mask.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]

    # RLE to string
    return ' '.join(str(x) for x in runs)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
import numpy as np
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.models import Model
from keras.optimizers import SGD
from keras.layers import Input, merge, concatenate, MaxPooling2D, UpSampling2D, Cropping2D, ZeroPadding2D
import pandas as pd
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, BatchNormalization, Activation, UpSampling2D
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import downscale_local_mean
from os.path import join
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D,MaxPool2D,MaxPool1D,Conv3D,MaxPool3D,MaxPooling2D,MaxPooling3D,Dense,Flatten,Reshape
from keras.layers import Conv2DTranspose
from keras.models import Sequential
import keras.backend as K
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from scipy.misc import imresize
df_mask = pd.read_csv(weg+'train_masks.csv', usecols=['img'])
ids_train = df_mask['img'].map(lambda s: s.split('_')[0]).unique()
from keras.metrics import binary_accuracy
imgs_idx = list(range(1, 17))
import cv2

In [ ]:
df_train = pd.read_csv(weg+'train_masks.csv')
ids_train = df_train['img'].map(lambda s: s.split('.')[0])


epochs = 100
batch_size = 1

ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.1)

In [ ]:
image_generator = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

mask_generator = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [ ]:
def batch_gen_2(size=1,height=160,width=240,aug=False,inter=False,fit=False):
    counter = 0 #счетчик запускам
    global reduced


    select = ids_train_split.values
    number_of_batches = np.ceil(len(select)/size) #количестов партий - зависит от размера партии
    while 1:



        cc = 0
        x = np.empty((size, height, width, 3), dtype=np.float32) # пустой массив для загрузки картинок
        y = np.empty((size, height, width, 1), dtype=np.float32)
        for img_id in select[size*counter:size*(counter+1)]: #16

            imgs_id = [cv2.resize(load_img(img_id),(width,height))/255.]
            # Input is image + mean image per channel + std image per channel
            mask_16 = [cv2.resize(load_mask(img_id), ( width,height))/ 255.]
            try:
                if mask_16[0].shape[2]>1:

                    mask_16 = [cv2.resize(load_mask_gif(img_id), ( width,height))/ 255.]
            except:
                pass            #X[i] = np.concatenate([imgs_id[idx-1], np.mean(imgs_id, axis=0), np.std(imgs_id, axis=0)], axis=2)
            
            x[cc:(1+cc)] = np.array(imgs_id)
            y[cc:(1+cc)] = np.expand_dims(np.array(mask_16),3)
            
            cc+=1    
        del imgs_id
        counter+=1
        SEED = int(np.random.randint(0,999999,1))
        if inter == False: #если не чередуем данные
            if aug == True:  # делаем ли аугментацию?
                if fit==True:
                    image_generator.fit(x)
                    mask_generator.fit(y)


                for img in image_generator.flow((x),batch_size=size,seed=SEED):
                    break

                for msk in mask_generator.flow((y),batch_size=size,seed=SEED):
                    break 
                yield(img,msk)

            if aug == False: # или базовую картинку?
                x = np.concatenate((x,model.predict(x)),axis =3)
                yield(x,y)

        if inter == True: # если чередуем то по очереди загружаем то одно то другое
            if counter % 2 ==0:
                if fit==True:
                    image_generator.fit(x)
                    mask_generator.fit(y)
                for img in image_generator.flow((x),batch_size=size,seed=SEED):
                    break

                for msk in mask_generator.flow((y),batch_size=size,seed=SEED):
                    break 
                yield(img,msk)

            else:
                yield(x,y)
                
        if counter == number_of_batches:
            counter = 0
            np.random.shuffle(select)

In [ ]:
def val_gen_2(size=1,height=160,width=240,aug=False,inter = False,fit=False):
    global reduced
    select = ids_valid_split.values
    number_of_batches = np.ceil(len(select)/size) #количестов партий - зависит от размера партии

    counter = 0
    while 1:



        cc = 0
        x = np.empty((size, height, width, 3), dtype=np.float32) # пустой массив для загрузки картинок
        y = np.empty((size, height, width, 1), dtype=np.float32)
        for img_id in select[size*counter:size*(counter+1)]: #16

            imgs_id = [cv2.resize(load_img(img_id),(width,height))/255.]
            # Input is image + mean image per channel + std image per channel
            mask_16 = [cv2.resize(load_mask(img_id), ( width,height))/ 255.]
            try:
                if mask_16[0].shape[2]>1:#если не одноканальный

                    mask_16 = [cv2.resize(load_mask_gif(img_id), ( width,height))/ 255.]
            except:
                pass            #X[i] = np.concatenate([imgs_id[idx-1], np.mean(imgs_id, axis=0), np.std(imgs_id, axis=0)], axis=2)
            
            x[cc:(1+cc)] = np.array(imgs_id)
            y[cc:(1+cc)] = np.expand_dims(np.array(mask_16),3)
            
            cc+=1    
        del imgs_id
        counter+=1
        
        SEED = int(np.random.randint(0,999999,1))
        if inter == False: #если не чередуем данные
            if aug == True:  # делаем ли аугментацию?
                if fit==True:
                    image_generator.fit(x)
                    mask_generator.fit(y)


                for img in image_generator.flow((x),batch_size=size,seed=SEED):
                    break
                for msk in mask_generator.flow((y),batch_size=size,seed=SEED):
                    break 
                yield(img,msk)
                #del img,msk
                #del x,y
            if aug == False: # или базовую картинку?
                x = np.concatenate((x,model.predict(x)),axis =3)
                yield(x,y)
                #del x,y
        if inter == True: # если чередуем то по очереди загружаем то одно то другое
            if counter % 2 ==0:
                if fit==True:
                    image_generator.fit(x)
                    mask_generator.fit(y)
                for img in image_generator.flow((x),batch_size=size,seed=SEED):
                    break
                
                for msk in mask_generator.flow((y),batch_size=size,seed=SEED):
                    break 
                yield(img,msk)
                #del img,msk
                #del x,y
            else:
                yield(x,y)
                #del x,y
        
        if counter == number_of_batches:
            counter = 0
            np.random.shuffle(select)

In [ ]:
for i in batch_gen_2(height=1280,width=1280):
    break
for j in val_gen_2(height=1280,width=1280):
    break    
print i[0].shape
print i[1].shape
print j[0].shape
print j[1].shape

In [ ]:
from renorm import BatchRenormalization
def get_unet_1024(input_shape=(1280, 1280, 4),
                 num_classes=1):
    inputs = Input(shape=input_shape)
    # 512

    down0a = Conv2D(16, (3, 3), padding='same')(inputs)
    down0a = BatchRenormalization()(down0a)
    down0a = ELU()(down0a)
    down0a = Conv2D(16, (3, 3), padding='same')(down0a)
    down0a = BatchRenormalization()(down0a)
    down0a = ELU()(down0a)
    down0a_pool = MaxPooling2D((2, 2), strides=(2, 2))(down0a)
    # 256

    down0 = Conv2D(32, (3, 3), padding='same')(down0a_pool)
    down0 = BatchRenormalization()(down0)
    down0 = ELU()(down0)
    down0 = Conv2D(32, (3, 3), padding='same')(down0)
    down0 = BatchRenormalization()(down0)
    down0 = ELU()(down0)
    down0_pool = MaxPooling2D((2, 2), strides=(2, 2))(down0)
    # 128

    down1 = Conv2D(64, (3, 3), padding='same')(down0_pool)
    down1 = BatchRenormalization()(down1)
    down1 = ELU()(down1)
    down1 = Conv2D(64, (3, 3), padding='same')(down1)
    down1 = BatchRenormalization()(down1)
    down1 = ELU()(down1)
    down1_pool = MaxPooling2D((2, 2), strides=(2, 2))(down1)
    # 64

    down2 = Conv2D(128, (3, 3), padding='same')(down1_pool)
    down2 = BatchRenormalization()(down2)
    down2 = ELU()(down2)
    down2 = Conv2D(128, (3, 3), padding='same')(down2)
    down2 = BatchRenormalization()(down2)
    down2 = ELU()(down2)
    down2_pool = MaxPooling2D((2, 2), strides=(2, 2))(down2)
    # 32

    down3 = Conv2D(256, (3, 3), padding='same')(down2_pool)
    down3 = BatchRenormalization()(down3)
    down3 = ELU()(down3)
    down3 = Conv2D(256, (3, 3), padding='same')(down3)
    down3 = BatchRenormalization()(down3)
    down3 = ELU()(down3)
    down3_pool = MaxPooling2D((2, 2), strides=(2, 2))(down3)
    # 16

    down4 = Conv2D(512, (3, 3), padding='same')(down3_pool)
    down4 = BatchRenormalization()(down4)
    down4 = ELU()(down4)
    down4 = Conv2D(512, (3, 3), padding='same')(down4)
    down4 = BatchRenormalization()(down4)
    down4 = ELU()(down4)
    down4_pool = MaxPooling2D((2, 2), strides=(2, 2))(down4)
    # 8

    center = Conv2D(1024, (3, 3), padding='same')(down4_pool)
    center = BatchRenormalization()(center)
    center = ELU()(center)
    center = Conv2D(1024, (3, 3), padding='same')(center)
    center = BatchRenormalization()(center)
    center = ELU()(center)
    # center

    up4 = UpSampling2D((2, 2))(center)
    up4 = concatenate([down4, up4], axis=3)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchRenormalization()(up4)
    up4 = ELU()(up4)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchRenormalization()(up4)
    up4 = ELU()(up4)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchRenormalization()(up4)
    up4 = ELU()(up4)
    # 16

    up3 = UpSampling2D((2, 2))(up4)
    up3 = concatenate([down3, up3], axis=3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchRenormalization()(up3)
    up3 = ELU()(up3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchRenormalization()(up3)
    up3 = ELU()(up3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchRenormalization()(up3)
    up3 = ELU()(up3)
    # 32

    up2 = UpSampling2D((2, 2))(up3)
    up2 = concatenate([down2, up2], axis=3)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchRenormalization()(up2)
    up2 = ELU()(up2)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchRenormalization()(up2)
    up2 = ELU()(up2)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchRenormalization()(up2)
    up2 = ELU()(up2)
    # 64

    up1 = UpSampling2D((2, 2))(up2)
    up1 = concatenate([down1, up1], axis=3)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchRenormalization()(up1)
    up1 = ELU()(up1)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchRenormalization()(up1)
    up1 = ELU()(up1)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchRenormalization()(up1)
    up1 = ELU()(up1)
    # 128

    up0 = UpSampling2D((2, 2))(up1)
    up0 = concatenate([down0, up0], axis=3)
    up0 = Conv2D(32, (3, 3), padding='same')(up0)
    up0 = BatchRenormalization()(up0)
    up0 = ELU()(up0)
    up0 = Conv2D(32, (3, 3), padding='same')(up0)
    up0 = BatchRenormalization()(up0)
    up0 = ELU()(up0)
    up0 = Conv2D(32, (3, 3), padding='same')(up0)
    up0 = BatchRenormalization()(up0)
    up0 = ELU()(up0)
    # 256

    up0a = UpSampling2D((2, 2))(up0)
    up0a = concatenate([down0a, up0a], axis=3)
    up0a = Conv2D(16, (3, 3), padding='same')(up0a)
    up0a = BatchRenormalization()(up0a)
    up0a = ELU()(up0a)
    up0a = Conv2D(16, (3, 3), padding='same')(up0a)
    up0a = BatchRenormalization()(up0a)
    up0a = ELU()(up0a)
    up0a = Conv2D(16, (3, 3), padding='same')(up0a)
    up0a = BatchRenormalization()(up0a)
    up0a = ELU()(up0a)
    # 512

    classify = Conv2D(num_classes, (1, 1), activation='sigmoid')(up0a)

    model = Model(inputs=inputs, outputs=classify)

    model.compile(optimizer='adam', loss=dice_coef_loss, metrics=[dice_loss,'accuracy'])
    #model.compile(optimizer=SGD(lr=0.01, momentum=0.9), loss=dice_coef_loss, metrics=[dice_loss,'accuracy'])
    #model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_loss,'accuracy'])
    #model.compile(optimizer=SGD(lr=0.01, momentum=0.9), loss='binary_crossentropy', metrics=[dice_loss,'accuracy'])
    #SGD(lr=0.01, momentum=0.9)
    return model



In [ ]:
model_rest = get_unet_1024()


In [ ]:
callbacks = [EarlyStopping(monitor='loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='loss',
                               factor=0.1,
                               patience=4,
                               cooldown=2,
                               verbose=1),
             ModelCheckpoint(filepath='restore_1280_1280_renorm_1_ELU.hdf5',monitor ='loss',
                             save_best_only=True,
                             save_weights_only=True,mode = 'min',verbose=1),
             TensorBoard(log_dir='logs')]
             
           
batch_size = 1


In [ ]:

model_rest.fit_generator(generator=batch_gen_2(size=batch_size,height=1280,width=1280),
        steps_per_epoch =int(len(ids_train_split)/batch_size),
        validation_data = val_gen_2(size=1,height=1280,width=1280),
        validation_steps=int(len(ids_valid_split)/batch_size),
        verbose=1,callbacks=callbacks,epochs =100)

In [ ]:
model_rest.load_weights('restore_1280_1280_renorm_1_ELU.hdf5')
model.load_weights(filepath='unet_new_1280_1280_2_batch_renorm_9_my_gen_ELU.hdf5')
model_rest.evaluate_generator(generator=batch_gen_2(size=batch_size,height=1280,width=1280),steps=np.ceil(float(len(ids_train_split)) / float(batch_size)))

In [ ]:
model_rest.evaluate_generator(generator=val_gen_2(size=1,height=1280,width=1280),steps=np.ceil(float(len(ids_train_split)) / float(batch_size)))

In [ ]:
import pandas as pd
import numpy as np

import cv2
from tqdm import tqdm


df_test = pd.read_csv(weg+'sample_submission.csv')
ids_test = df_test['img'].map(lambda s: s.split('.')[0])

input_size = 1280
batch_size = 1

orig_width = 1918
orig_height = 1280

threshold = 0.5

#model = get_unet_1024()
#model.load_weights(filepath='unet_new_1280_1280_2_batch_renorm.hdf5')

names = []
for id in ids_test:
    names.append('{}.jpg'.format(id))


# https://www.kaggle.com/stainsby/fast-tested-rle
def run_length_encode(mask):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    inds = mask.flatten()
    runs = np.where(inds[1:] != inds[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    rle = ' '.join([str(r) for r in runs])
    return rle


rles = []

test_splits = 8*59  # Split test set (number of splits must be multiple of 2)
ids_test_splits = np.split(ids_test, indices_or_sections=test_splits)



In [ ]:
def _mask_to_rle_string(mask):
    """Convert boolean/`binary uint` mask to RLE string."""
    # Mask to RLE
    pixels = mask.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]

    # RLE to string
    return ' '.join(str(x) for x in runs)

In [ ]:
split_count = 0
for ids_test_split in ids_test_splits:
    split_count += 1


    def test_generator_2():
        while True:
            input_size = 1280
            for start in range(0, len(ids_test_split), batch_size):
                x_batch = []
                end = min(start + batch_size, len(ids_test_split))
                ids_test_split_batch = ids_test_split[start:end]
                for id in ids_test_split_batch.values:
                    img = cv2.imread(weg+'test_old/{}.jpg'.format(id))
                    img = cv2.resize(img, (input_size, input_size))
                    x_batch.append(img)
                x_batch = np.array(x_batch, np.float32) / 255
                x_batch = np.concatenate((x_batch,model.predict(x_batch)),axis =3)
                yield x_batch


    print("Predicting on {} samples (split {}/{})".format(len(ids_test_split), split_count, test_splits))
    preds = model_rest.predict_generator(generator=test_generator_2(),
                                    steps=np.ceil(float(len(ids_test_split)) / float(batch_size)))
    preds = np.squeeze(preds, axis=3)

    print("Generating masks...")
    for pred in tqdm(preds, miniters=1000):
        prob = cv2.resize(pred, (orig_width, orig_height))
        mask = prob > threshold
        rle = _mask_to_rle_string(mask)
        rles.append(rle)

print("Generating submission file...")
df = pd.DataFrame({'img': names, 'rle_mask': rles})
df.to_csv('submit/restore_1280_1280_renorm_1_ELU_old_pics.csv.gz', index=False, compression='gzip')
print 'finished'